Please remember to update the token. For multiplayer to work, each player needs a different token. Please feel free to use multiple tokens in the same group.

In [13]:
#Update your token
STUDENT_TOKEN = 'GROUP 3'

There should be no need to change the code below.

In [14]:
## ignore this code, just used for submission
import requests
import pprint
import json
import random
import time
from copy import copy, deepcopy

class Game:
  def __init__(self, state, status, player):
    self.state = state
    self.status = status
    self.player = player

  def is_waiting(self):
    return self.status == 'waiting'

  def is_end(self):
    return self.status == 'complete'
  
  def get_board(self):
    return json.loads(self.state)

  def get_winner(self):
    return None

  def actions(self):
    return []

  def print(self):
    print(self.state)

def new_game(game_type, multi_player = False):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/new-game?TOKEN=%s&game-type=%s&multi-player=%s' % (STUDENT_TOKEN, game_type, 'True' if multi_player else 'False'))
    if r.status_code == 200:
      return r.json()['game-id']
    print(r.content)

def join_game(game_type, game_id):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/join-game?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return r.json()['player']
    print(r.content)

def game_state(game_type, game_id, GameClass):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/game-state?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return GameClass(r.json()['state'], r.json()['status'], r.json()['player'])
    print(r.content)

def update_game(game_type, game_id, player, move):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/update-game?TOKEN=%s&game-type=%s&game-id=%s&player=%s&move=%s' % (STUDENT_TOKEN, game_type, game_id, player, move))
    if r.status_code == 200:
      return r.content
    print(r.content)

def game_loop(solver, GameClass, game_type, multi_player = False, id = None):
  while id == None:
    print('Creating new game...')
    id = new_game(game_type, multi_player)

  print('Joining game with id: %s' % id)
  player = join_game(game_type, id)

  print('Playing as %s' % player)

  game = game_state(game_type, id, GameClass)
  print('Waiting for the other player to join...')
  while game.is_waiting():
    time.sleep(10)
    game = game_state(game_type, id, GameClass)

  while True:
    game = game_state(game_type, id, GameClass)
    game.print_game()
    if game.is_end():
      if game.player == '-':
        print('draw')
      else:
        print('winner' if game.player == player else 'loser')
      return
    if game.player == player:
      print('Making next move...')
      move = solver(game)
      update_result = update_game(game_type, id, player, json.dumps(move))
      print(update_result)
    else:
      time.sleep(2)

There is almost certainly no need to change this code, but if you must you can.

In [15]:
from functools import reduce
from copy import copy, deepcopy
import json
import random

class ConnectFour(Game):
  def __init__(self, state, status, player):
    Game.__init__(self, state, status, player)

  def actions(self):
    board = self.get_board()
    legalMoves = [] 
    for col in range(0, len(board[0])):
        if board[0][col] == '.':
          legalMoves.append(col)
    return legalMoves

  def get_winner(self):
    return self.winner # this should return the actual winner

  def other_player(self):
    if self.player == 'O': return 'X'
    if self.player == 'X': return 'O'

  def print_game(self):
    print(self.state)

Instead of changing this code for a random solver, write new code that also returns the next move. For example, create a new "amazing_solver"

In [16]:
def random_solver(game):
  board = game.get_board()
  while True:
      x = 0
      y = random.randint(0, len(board[0]) - 1)
      if board[x][y] == '.':
        return y
  return None

This code is the one that starts the game. In different games you will need to change the class (ConnectFour is the one for connect four game). In connect four, there are two game types, 'connect4' for a small board, and 'connect4big' for a big board.

The 'multi_player' variable can be True or False. If you set multi_palyer to True, you will need to provide a game_id against which to play or if using None a new game will be created (which another player can join).

In [17]:
#game_loop(random_solver, ConnectFour, 'connect4big', multi_player=False, id=None)

In [18]:
def getMaximiserMoves(board, maximiser, minimiser):

  """
  
  This function is used to get the best moves to help the maximiser win.

  Parameters:
    board - game board
    maximiser - symbol of 1st player (who we want to win)
    minimiser - symbol of 2nd player (who we want to lose) 

  Return:
    tuple with the best moves for the maximiser to win in the form of 3 arrays:
    win array that allows for the maximiser to connect 4, lose array that allows
    for the maximiser to block the minimiser from connecting 4 and best which are
    the next best moves. The tuple and arrays are ordered by priority 

  """

  #get spaces where maximiser may win a row
  # board has 7 column so we will divide each row into 

  # Storing all rows
  allRows = []

  for row in board:
    allRows.append(row)
  
  # Storing all columns
  allColumns = []
  for i in range(len(board[0])):
    col = []
    for row in allRows:
      col.append(row[i])
    allColumns.append(col)

  win = [] # stores moves that allow maximiser to connect 4
  block = [] # stores moves that allow maximiser to block minimiser from winning
  best = [] # stores next best moves for maxmiser (connect 3 and 2)

  rowsOf4 = [] # stores 4 adjacent row spaces for all rows
  rowsOf3 = [] # stores 3 adjacent row spaces for all rows
  rowsOf2 = [] # stores 2 adjacent row spaces for all rows
  
  colsOf4 = [] # stores 4 adjacent column spaces for all columns
  colsOf3 = [] # stores 3 adjacent column spaces for all columns
  colsOf2 = [] # stores 2 adjacent column spaces for all columns
  

  # goes through each row
  for rowIndex in range(len(allRows)):

    # gets current row
    row = allRows[rowIndex]

    # stores an array of 4 adjacent row spaces from all rows
    rowsOf4 = []
    rowsOf4.append([row[0:4]])
    rowsOf4.append([row[1:5]])
    rowsOf4.append([row[2:6]])
    rowsOf4.append([row[3:7]])

    # loops through each array of 4 adjacent rows to 
    for r in rowsOf4:

      # finds a space where the maximiser would be able to connect 4 and adds 
      # that column to the win array which takes highest priority 
      if r.count(maximiser) == 3 and r.count(".") == 1:
        win.append(r.index("."))
      
      # finds a space where the minimiser would be able to connect 4 and adds 
      # that column to the block array which takes 2nd highest priority 
      if r.count(minimiser) == 3 and r.count(".") == 1:
        block.append(r.index("."))
      

    # stores an array of 3 adjacent row spaces from all rows
    rowsOf3 = []
    rowsOf3.append([row[0:3]])
    rowsOf3.append([row[1:4]])
    rowsOf3.append([row[2:5]])
    rowsOf3.append([row[3:6]])
    rowsOf3.append([row[4:7]])

    # loops through each space of 3 adjacent rows to find a space where the
    # maximiser would be able to connect 3 and adds that the best array which 
    # takes 3rd highest priority

    for r in rowsOf3:
      if r.count(maximiser) == 2 and r.count(".") == 1:
        best.append(r.index("."))

    # stores an array of 2 adjacent row spaces for all rows
    rowsOf2 = []
    rowsOf2.append([row[0:2]])
    rowsOf2.append([row[1:3]])
    rowsOf2.append([row[2:4]])
    rowsOf2.append([row[3:5]])
    rowsOf2.append([row[4:6]])
    rowsOf2.append([row[5:7]])

    # loops through each space of 2 adjacent rows to find a space where the
    # maximiser would be able to connect 2 and adds that the best array which 
    # takes 3rd highest priority
    for r in rowsOf2:
      if r.count(maximiser) == 1 and r.count(".") == 1:
        best.append(r.index("."))


  # goes through each column
  for colIndex in range(len(allColumns)):

    # gets current column
    col = allColumns[colIndex]

    # stores an array of 4 adjacent column spaces from all columns
    colsOf4 = []
    colsOf4.append([col[0:4]])
    colsOf4.append([col[1:5]])
    colsOf4.append([col[2:6]])

    # loops through each array of 4 adjacent columns
    for c in colsOf4:

      # finds a space where the maximiser would be able to connect 4 and adds 
      # that column to the block array which takes highest priority 
      if c.count(maximiser) == 3 and c.count(".") == 1:
        win.append(colIndex)

      # finds a space where the maximiser would be able to connect 4 and adds 
      # that column to the win array which takes 2nd highest priority 
      if c.count(minimiser) == 3 and c.count(".") == 1:
        block.append(colIndex)

    # stores an array of 3 adjacent column spaces from all columns
    colsOf3 = []
    colsOf3.append([col[0:3]])
    colsOf3.append([col[1:4]])
    colsOf3.append([col[2:5]])
    colsOf3.append([col[3:6]])

    # loops through each space of 3 adjacent rows to find a space where the
    # maximiser would be able to connect 3 and adds that the best array which 
    # takes 3rd highest priority
    for s in colsOf3:
      if c.count(maximiser) == 2 and c.count(".") == 1:
        best.append(colIndex)


    # stores an array of 2 adjacent column spaces from all columns
    colsOf2 = []
    colsOf2.append([col[0:2]])
    colsOf2.append([col[1:3]])
    colsOf2.append([col[2:4]])
    colsOf2.append([col[3:5]])
    colsOf2.append([col[4:6]])

    # loops through each space of 2 adjacent rows to find a space where the
    # maximiser would be able to connect 2 and adds that the best array which 
    # takes 3rd highest priority
    for c in colsOf2:
      if c.count(maximiser) == 1 and c.count(".") == 1:
        best.append(rowIndex)

  
  # returns a tuple 
  return (win, block, best)

In [19]:
from random import randint
def getCutoffMoves(maximiserMoves, legalMoves, isMaximiser):

  """
  
  We use a cutoff function to only explore nodes that seem promising for the maximiser

  Parameters:
    maximiserMoves - tuple with array of various moves: win that would help the 
    maximiser to connect 4), block that prevents the minimiser from connecting 4 
    and best which helps the maximiset to connect 3 or 2

    legalMoves - contains all possible moves regardless of how how useful they are
    to the maximiser or minimiser

    isMaximiser - Boolean value to indicate whether or not the current player is 
    the maximiser or minimiser

  Return:
    an array with the relevant cutoff moves for each player


  """

  # We use cutoff so as to not search all the child moves

  win = maximiserMoves[0] # moves that will help maximiser connect 4 immediately
  block = maximiserMoves[1] # moves that will help maximiser immediately block minimiser from connecting 4
  best = maximiserMoves[2] # moves that will help player connect 2 or 3

  if isMaximiser: # here we try to get the moves that help the maximiser win
    if len(win) > 0:
        return win # returns win array if it is not empty
    elif len(block) > 0:
      return block # returns block array if it is not empty
    elif len(best) > 0:
      return best # returns best array if it is not empty
    else: # else it returns the legal moves
      # if the board starts off empty, we return an array with just the center
      # piece move as that is assumed to be the best, else we just return all
      # the other legal moves
      if len(legalMoves) == 0: 
        return[legalMoves[int(len(legalMoves) / 2)]]
      return legalMoves

  else: # here we try to get the moves to help the maximiser lose
    worst = [] # initialise array to store worst moves
    maximiserMoves = win + block + best # get all the maximiser moves
    
    # we loop through the legal moves array to only append moves to the worst 
    # array that would not help the minimiser; then we return that array
    if len(maximiserMoves) > 0:
      for m in legalMoves:
        if m not in maximiserMoves:
          worst.append(m)
    return worst

In [20]:
def getMoveValue(game, move, player):

  """

  This function simulates a move to see if it leads to a win or not.

  Parameter:
    game - current game
    move - column number obtained for maximiser and minimiser based on best move
    player - symbol for player making move

  Return
    1 if move is winning move or 0 if move is losing move

  """

  gameTemp = deepcopy(game) # copy of game
  boardTemp = gameTemp.get_board() # copy of board

  # looping through rows from bottom to top to fill empty space for player
  for i in (len(boardTemp)-1, -1, -1):
    if boardTemp[i][move] == ".":
      boardTemp[i][move] = player
  
  if gameTemp.is_end(): # checks if move brought game to an end
    return 1 # changes value to 1 if the move is a winning move
  else:
    return 0 # changes value to 0 if the move is a losing move

In [21]:
def minimax(move, value, game, depth, alpha, beta, maximiser, minimiser): 

  """
  The minimax algorithm is a depth-first search algorithm that is commonly used
  to solve two player games -  in this case Connect 4. The game is simple - 
  two players get take turns dropping a circle in an empty slot of a suspend 
  grid. Whoever gets four cricles in a row (horizontally, vertically or 
  diagonally), wins the game. There are many different variations of the suspend 
  grid's dimensions such as 7 column × 6 rows, 5×4, 6×5, 8×7, 9×7, 10×7 and 8×8.

  The maximiser starts the game and tries to make a move that will give him the 
  best outcome.

  The minimiser is the opponent who tries to make the worst move as we are 
  rooting for the maximiser to win the game.

  This minimax algorithm is made better with alpha-beta pruning as it reduces
  the number of possible moves to explore by elimination any move that would 
  lead to a worse outcome than a previous move for the maximiser. We also cut
  off the child expansion for promising nodes for the maximiser and non-promising 
  nodes for the minimiser.

  This minimax algorithm aims to always ensure that the maximiser wins.

  Parameters:
    move - a column index being examined where there are empty spaces in
    that column.
    game - the game intitalised by the server
    depth - the intial lenght of the board in terms of number of rows
    alpha - positive infinity
    beta - negative infinity
    maximiser - player we hope to win 
    minimiser - player we hope to lose

  Return:
    move - the most optimal move (column index) calculated for the current 
    player if it is the or the least optimal move calculated for the current 
    player if it is the minimiser

  """
  board = game.get_board() # game state with rows and columns and the pieces dropped
  legalMoves = game.actions() # we use the actions function in the ConnectGame
  # class to get an array with the indices of all columns with and empty space

  # if the search depth has been exceeded or a winner has been found or there
  # are no more empty slots, we return the move of the last player
  if depth == 0 or game.is_end() or len(legalMoves) == 0:
    return move
    
  if game.player == maximiser: # current player is maximiser  
    bestValue = negInf # minimum score of which the maximiser is assured   
    maximiserMoves = getMaximiserMoves(board, maximiser, minimiser)
    # we only explore moves that we believe would best help the maximiser to win
    for move in getCutoffMoves(maximiserMoves, legalMoves, isMaximiser = True):
      value = minimax(move, value, game, depth-1, alpha, beta, maximiser, minimiser) 
      bestValue = max(bestValue, alpha)
      alpha = max(bestValue, alpha)
      if alpha >= beta:
        break  
      
  else: # current player is minimiser 
    bestScore = posInf # maximum score of which the minimiser is assured 
    maximiserMoves = getMaximiserMoves(board, maximiser, minimiser)
    # explore all cutoff moves for the minimiser (we already removed those
    # moves that would help the minimiser to win)
    for move in getCutoffMoves(maximiserMoves, legalMoves, isMaximiser = False):
      value = minimax(move, value, game, depth-1, alpha, beta, maximiser, minimiser)  
      bestScore = min(bestScore, beta) 
      beta = min(bestScore, beta)
      if alpha >= beta:  
        break   

  # simulates adding move to board, returns 1 if it is a winning move or 0 if
  # it is a losing move
  return getMoveValue(game, move, game.player) 

In [22]:
from ctypes import memmove
from IPython.utils.path import get_long_path_name



# We need to get the values for infinity
# alpha represents positive infinity
# beta represents negative infinity
import math
posInf = math.inf
negInf = -math.inf

def minimaxSolver(game):

  """

  This function calls an instance of the aglorithm that server takes to solve 
  the game.

  Parameters:
    game - the game generated by the server

  Return:
    move - the column index where a player in a round (game loop) drops a piece
    based on the best piece predicted by the algorithm
  """ 
  
  board = game.get_board() # stores the game state with rows and columns with their pieces dropped
  depth = len(board) # the depth/row count of the board is important or else maximum recursion will ensue
  breadth = len(board[0]) # column count
  maximiser = game.player # assumes that the first player is the maximiser we hope to win
  minimiser = game.other_player() # assumes that the other player is the minimiser we hope to lose

  # The algorithm we use for minimax passes the move (column index) as a parameter. Inside the function itself, the move is 
  # obtained by looping through the list of legal column indices (a column with an empty space). Given that we would not be 
  # able to get the value for this move when calling the function itself (as the calculation is made inside the function), we
  # will just set an initial value of move (which will get a true value calcuated inside the function).
  # We will just set the initial move to the middle column index as that is usually the safest startin position
  # in a real game of connect 4.
  initialMove = int((len(board[0]))/2)

  # this code calls the function for the actual minimax algorithm which runs
  # recursively
  move = minimax(initialMove, 0, game, depth, negInf, posInf, maximiser, minimiser) 
  return move # final move assumed to be the best (for maximiser) or worst (for minimiser is returned)

In [23]:
game_loop(minimaxSolver, ConnectFour, 'connect4big', multi_player=False, id=None)

Creating new game...
Joining game with id: 10047
Playing as O
Waiting for the other player to join...
[[".", ".", ".", ".", ".", ".", "."], [".", ".", ".", ".", ".", ".", "."], [".", ".", ".", ".", ".", ".", "."], [".", ".", ".", ".", ".", ".", "."], [".", ".", ".", ".", ".", ".", "."], [".", ".", ".", "X", ".", ".", "."]]
Making next move...
b'Valid move'
[[".", ".", ".", ".", ".", ".", "."], [".", ".", ".", ".", ".", ".", "."], [".", ".", ".", ".", ".", ".", "."], [".", ".", ".", ".", ".", ".", "."], [".", ".", ".", "X", ".", ".", "."], ["O", ".", ".", "X", ".", ".", "."]]
Making next move...
b'Valid move'
[[".", ".", ".", ".", ".", ".", "."], [".", ".", ".", ".", ".", ".", "."], [".", ".", ".", ".", ".", ".", "."], [".", ".", ".", ".", ".", ".", "."], ["O", ".", ".", "X", ".", ".", "."], ["O", ".", ".", "X", ".", ".", "X"]]
Making next move...
b'Valid move'
[[".", ".", ".", ".", ".", ".", "."], [".", ".", ".", ".", ".", ".", "."], [".", ".", ".", ".", ".", ".", "."], ["O", ".", ".",